## Examples using `ase-weas-widget`

Installing `ase-weas-widget` registers `weas-widget` as an external viewer for `ase.visualize.view`.  
NOTE: We require ase >= 3.24 as the entry point system is only added at this release

In [1]:
from ase.build import bulk, molecule
from ase.visualize import view

In [2]:
atoms = bulk('Si', 'diamond', 5.4)
view(atoms.repeat((2,2,2)), viewer='weas')

WeasWidget(children=(BaseWidget(atoms={'species': {'Si': 'Si'}, 'cell': [0.0, 5.4, 5.4, 5.4, 0.0, 5.4, 5.4, 5.…

The view can be customized via a mods function

In [3]:
def apply_vesta(viewer):
    """mods to customize the WeasWidget object"""
    viewer.avr.model_style = 1
    viewer.avr.show_bonded_atoms = True
    viewer.avr.color_type = "VESTA"
    
atoms = bulk('Si', 'diamond', 5.4)
view(atoms.repeat((2,2,2)), viewer='weas', mods=apply_vesta)

WeasWidget(children=(BaseWidget(atoms={'species': {'Si': 'Si'}, 'cell': [0.0, 5.4, 5.4, 5.4, 0.0, 5.4, 5.4, 5.…

The mods function is actually unnecesssary, since we can modifer the returned `WeasWidget` object directly...  
This works since the notebook will just show the last returned object

In [4]:
atoms = bulk('Si', 'diamond', 5.4)
viewer = view(atoms.repeat((2,2,2)), viewer='weas')
viewer.avr.model_style = 1
viewer.avr.show_bonded_atoms = True
viewer.avr.color_type = "VESTA"
viewer

WeasWidget(children=(BaseWidget(atoms={'species': {'Si': 'Si'}, 'cell': [0.0, 5.4, 5.4, 5.4, 0.0, 5.4, 5.4, 5.…

Or with a simple modifier string, the avaliable options can be found at https://github.com/bonan-group/ase-weas-widget/blob/main/src/ase_weas_widget/viewer.py

In [5]:
view(atoms.repeat((2,2,2)), viewer='weas', presets='vesta,ball,legend')  # Apply VESTA style then switch to ball

WeasWidget(children=(BaseWidget(atoms={'species': {'Si': 'Si'}, 'cell': [0.0, 5.4, 5.4, 5.4, 0.0, 5.4, 5.4, 5.…

In [6]:
atoms = bulk('MgO', 'rocksalt', 4.0)
view(atoms.repeat((2,2,2)), viewer='weas')

WeasWidget(children=(BaseWidget(atoms={'species': {'Mg': 'Mg', 'O': 'O'}, 'cell': [0.0, 4.0, 4.0, 4.0, 0.0, 4.…

Play a trajectory 

In [7]:

atoms = molecule("C2H6SO")
# create a list of atoms by rotating it.
images = []
for i in range(0, 360, 10):
    new_atoms = atoms.copy()
    new_atoms.rotate("z", i)
    images.append(new_atoms)
view(images, viewer='weas')

WeasWidget(children=(BaseWidget(atoms=[{'species': {'S': 'S', 'O': 'O', 'C': 'C', 'H': 'H'}, 'cell': [0.0, 0.0…

In [8]:
import numpy as np
atoms = bulk("Fe", cubic=True)
atoms*=[2, 2, 1]
atoms.set_array("moment", np.ones(len(atoms)))
def show_moment(viewer):
    viewer.avr.model_style = 1
    viewer.camera.setting = {"direction": [0, -1, 0]}
    return viewer
    
view(atoms, mods=show_moment, viewer='weas')



WeasWidget(children=(BaseWidget(atoms={'species': {'Fe': 'Fe'}, 'cell': [5.74, 0.0, 0.0, 0.0, 5.74, 0.0, 0.0, …

Add custom vector field

In [9]:
# origin of the vector
origins = atoms.positions
# the vertor
vectors = [[1, 1, 1]]*len(atoms)

def add_vector(viewer):
    viewer.avr.model_style = 1
    viewer.camera.setting = {"direction": [0, -1, 0]}
    viewer.avr.vf.settings["test"]= {"origins": origins, "vectors": vectors, "color": "red"}
    return viewer
    
view(atoms, mods=add_vector, viewer='weas')


WeasWidget(children=(BaseWidget(atoms={'species': {'Fe': 'Fe'}, 'cell': [5.74, 0.0, 0.0, 0.0, 5.74, 0.0, 0.0, …

Color by attribute

In [10]:
import numpy as np
from ase.build import bulk

atoms = bulk("Fe", cubic=True)

def apply_phonon(viewer):
    phonon_setting = {"eigenvectors": np.array([[[0, 0], [0, 0],[0.5, 0]],
                                        [[0, 0], [0, 0], [-0.5, 0]]]
                                       ),
            "kpoint": [0, 0, 0], # optional
            "amplitude": 5, # scale the motion of the atoms
            "factor": 1.5, # scale the length of the arrows
            "nframes": 20,
            "repeat": [4, 4, 1],
            "color": "blue",
            "radius": 0.1,
            }
    
    viewer.avr.phonon_setting = phonon_setting
    viewer.camera.setting = {"direction": [0, -1, 0]}

view(atoms, mods=apply_phonon, viewer='weas')

WeasWidget(children=(BaseWidget(atoms={'species': {'Fe': 'Fe'}, 'cell': [2.87, 0.0, 0.0, 0.0, 2.87, 0.0, 0.0, …

Color by attribute - here we set the color via forces

In [11]:
from ase.build import bulk
from ase.calculators.emt import EMT
import numpy as np
from weas_widget import WeasWidget

atoms = bulk('Au', cubic = True)
atoms *= [3, 3, 3]
atoms.positions += np.random.random((len(atoms), 3))
atoms.calc = EMT()
atoms.get_potential_energy()
# set the forces as an attribute
atoms.set_array("Force", atoms.calc.results["forces"])

def color_by_forces(viewer):
    viewer.avr.color_by = "Force"
    viewer.avr.color_ramp = ["red", "yellow", "blue"]
    viewer.avr.model_style = 1
    viewer
view(atoms, viewer='weas', mods=color_by_forces)

WeasWidget(children=(BaseWidget(atoms={'species': {'Au': 'Au'}, 'cell': [12.24, 0.0, 0.0, 0.0, 12.24, 0.0, 0.0…

Customize the viewer

In [12]:
atoms = bulk('MgO', 'rocksalt', 4.0)
viewerStyle = {"width": "600px", "height": "600px"}
guiConfig={
           "components": {
                          "enabled": True,
                          "atomsControl": True,
                          "buttons": True},
           "buttons": {
                       "enabled": True,
                       "fullscreen": True,
                       "download": True,
                       "measurement": True,
                       }
         }
view(atoms.repeat((2,2,2)), viewer='weas', viewer_kwargs={'viewerStyle': viewerStyle, 'guiConfig': guiConfig}, 
    mods=lambda x: setattr(x.avr, 'model_style', 1) 
    )



WeasWidget(children=(BaseWidget(atoms={'species': {'Mg': 'Mg', 'O': 'O'}, 'cell': [0.0, 4.0, 4.0, 4.0, 0.0, 4.…